# AR simulation

### 1. Imports and constants

In [1]:
import copy

import numpy as np
%load_ext autoreload
%autoreload 2

# Add project to path
import os
import sys

import pandas as pd
from lib.generate.generate_ar import generate_stationary_ar_coefficients, simulate_ar
from lib.dataprocessor.ArDataProcessor import ArDataProcessor
from lib.models.ArModale import ArModel
from lib.loss.Mse import Mse

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# set constants
length = 1_000
train_ratio = 0.06
test_ratio = 0.04
theoretical_risk_estimator_ratio = 0.9
max_degree = 100
sigma = 1

data_processor = ArDataProcessor('ar', max_degree)
models = [ArModel(i) for i in range(1, max_degree)]
loss = Mse()

In [ ]:
def run_one_simulation(degree, seed):
    # generate data
    ar_coefficients = generate_stationary_ar_coefficients(degree=degree, seed=seed)
    series = simulate_ar(ar_coefficients, sigma, length, seed=seed)
    
    # process data
    x, y = data_processor.process(series)
    
    # split data
    train_size = int(length * train_ratio)
    test_size = int(length * test_ratio)
    theoretical_risk_estimator_size = int(length * theoretical_risk_estimator_ratio)
    x_train = x[:train_size]
    y_train = y[:train_size]
    x_test = x[train_size:train_size + test_size]
    y_test = y[train_size:train_size + test_size]
    x_theoretical_risk_estimator = x[-theoretical_risk_estimator_size:]
    y_theoretical_risk_estimator = y[-theoretical_risk_estimator_size:]
    
    test_values_and_predictions = pd.DataFrame(data=y_test, columns=['y'])
    theoretical_risk_estimator_values_and_predictions = pd.DataFrame(data=y_theoretical_risk_estimator, columns=['y'])
    
    # fit and predict
    for model in models:
        model = model.fit(x_train, y_train)
        test_values_and_predictions[model.name] = model.predict(x_test)
        theoretical_risk_estimator_values_and_predictions[model.name] = model.predict(x_theoretical_risk_estimator)
        
    # oracle predictions
    oracle = ArModel(degree)
    oracle = oracle.force_coef(ar_coefficients)
    
    test_values_and_predictions['oracle'] = oracle.predict(x_test)
    theoretical_risk_estimator_values_and_predictions['oracle'] = oracle.predict(x_theoretical_risk_estimator)
    
    # loss
    test_loss = loss.compute_from_dataframe(test_values_and_predictions, [model.name for model in models])
    theoretical_risk = loss.compute_from_dataframe(theoretical_risk_estimator_values_and_predictions, [model.name for model in models]).mean(axis=0)
    
    # select best model
    non_oracle_models_loss = test_loss[[model.name for model in models]]
    best_model_theoretical_risk = []
    for m in range(1, test_size):
        sub_test_loss = non_oracle_models_loss[:m]
        best_model = sub_test_loss.mean(axis=0).idxmin()
        best_model_theoretical_risk.append(copy.deepcopy(theoretical_risk.loc[best_model]))
    
    return best_model_theoretical_risk - theoretical_risk.loc['oracle']

theoretical_risk_diffs = []
degree = 20
for seed in range(1, 20):
    theoretical_risk_diffs.append(run_one_simulation(degree, seed))
mean_best_model_theoretical_risk = np.array(theoretical_risk_diffs).mean(axis=0)

In [ ]:
# plot best_model_theoretical_risk
import matplotlib.pyplot as plt

plt.plot(mean_best_model_theoretical_risk)
plt.title(f'Error Differences between Best Model and Oracle')
plt.xlabel('Test Size')
plt.ylabel('Error Difference')
plt.legend()
plt.grid(True)
plt.show()